In [1]:
!pip install openpyxl

In [1]:
import matplotlib.pyplot as plt
import category_encoders as ce
from preprocessing import trainTestClean
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ridge_regression
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import pgeocode
import numpy as np

import xgboost as xgb
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor

import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_train, df_test = trainTestClean()

X_train = df_train.drop(columns=['price'])
y_train = df_train['price']
X_test = df_test.drop(columns=['price'])
y_test = df_test['price']

X_test = X_test[X_train.columns]

In [8]:
# select multiple models

models = {
    #'LinearRegression': LinearRegression(),
    #'Lasso': linear_model.Lasso(alpha=0.1),
    #'DecisionTree': DecisionTreeRegressor(random_state=42),
    #'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    #'ElasticNet': ElasticNet(random_state=0),
    'XGBoost': xgb.XGBRegressor(n_estimators=2000, random_state=42, learning_rate=0.1),
    'XGBoostElsa': xgb.XGBRegressor(n_estimators=2000, random_state=42, learning_rate=0.05, subsample= 0.8),
    #'XGBoostAlex': xgb.XGBRegressor(n_estimators=2500, random_state=42, learning_rate=0.08, subsample= 0.8),
    #'XGBoostAlex2': xgb.XGBRegressor(n_estimators=2500, random_state=42, learning_rate=0.08),
    'XGBoostGridCV': xgb.XGBRegressor(n_estimators=3000, model__max_depth=7,random_state=42, learning_rate=0.01, subsample= 0.8),
    'XGBoostBrutForce': xgb.XGBRegressor(colsample_bylevel=0.9289879319689553, colsample_bytree=0.7245003417617129, learning_rate=0.05183941032332593, max_depth=9, n_estimators=2496, reg_alpha=1.9905053073241674, reg_lambda=0.05061583846218687, subsample=0.7482424154252496),
    #'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.2),
    #'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    #'AdaBoost': AdaBoostRegressor(random_state=42),
    #'LightGBM': lgb.LGBMRegressor(random_state=42),
    #'CatBoost': CatBoostRegressor(random_state=42, silent=True),
    #'Ridge': Ridge(alpha=1.0),
    #'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
}

results = {}
best_mae = float('inf')
best_model_name = ''
best_pipeline = Pipeline([])
print("Colonnes de X_train :", X_train.columns.tolist())
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])

    pipeline.fit(X_train.drop(columns='id'), y_train)

    preds = pipeline.predict(X_test.drop(columns='id'))
    
    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)

    errors = abs(preds - y_test)
    mape = 100 * (errors / y_test)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print(f"{name} : MAE = {mae:.4f}, MSE = {mse:.4f}, accuracy = {accuracy:.4f}")

    results[name] = mae

    if mae < best_mae:
        best_mae = mae
        best_mse = mse
        best_accuracy = accuracy
        best_model_name = name
        best_pipeline = pipeline
        best_model = model

print("Models results :")
for model_name, mae in results.items():
    print(f"{model_name} : MAE = {mae:.4f}ler")

print(f"\n -> Best Model : {best_model_name} with MAE = {best_mae:.4f} and MSE = {best_mse:.4f}; accuracy = {best_accuracy:.4f}")

Colonnes de X_train : ['id', 'bedroomCount', 'bathroomCount', 'postCode', 'habitableSurface', 'hasAttic', 'hasBasement', 'buildingCondition', 'buildingConstructionYear', 'hasLift', 'floodZoneType', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasThermicPanels', 'landSurface', 'hasLivingRoom', 'livingRoomSurface', 'hasGarden', 'gardenSurface', 'parkingCountIndoor', 'parkingCountOutdoor', 'hasAirConditioning', 'hasArmoredDoor', 'hasVisiophone', 'hasOffice', 'toiletCount', 'hasSwimmingPool', 'hasFireplace', 'hasTerrace', 'terraceSurface', 'epcScore', 'facadeCount', 'isHouse', 'province_Antwerp', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Limburg', 'province_Liège', 'province_Luxembourg', 'province_Namur', 'province_Walloon Brabant', 'province_West Flanders', 'heating_CARBON', 'heating_ELECTRIC', 'heating_FUELOIL', 'heating_GAS', 'heating_PELLET', 'heating_SOLAR', 'heating_WOOD', 'kitchen_HYPER_EQUIPPED', 'kitchen_INSTALLED', 'kitch

In [ ]:
model = best_model

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
])

pipeline.fit(X_train.drop(columns='id'), y_train)
preds = pipeline.predict(X_test.drop(columns='id'))

mae = mean_absolute_error(y_test, preds)
errors = abs(preds - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)

print(f"{best_model_name} : MAE = {mae:.4f}, accuracy = {accuracy:.4f}")


ValueError: X does not contain the columns listed in cols

In [ ]:

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', xgb.XGBRegressor(random_state=42))
])

param_grid = {
    'model__n_estimators': [1000, 2000, 3000],
    'model__max_depth': [5, 7, 9],
    'model__learning_rate': [0.005, 0.01, 0.1, 0.2],
    'model__subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

grid_search.fit(X_train.drop(columns='id'), y_train)
print("Best params:", grid_search.best_params_)

preds = grid_search.predict(X_test.drop(columns='id'))

mae = mean_absolute_error(y_test, preds)
errors = abs(preds - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)

print(f"{best_model_name} : MAE = {mae:.4f}, accuracy = {accuracy:.4f}")

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best params: {'model__learning_rate': 0.01, 'model__max_depth': 7, 'model__n_estimators': 3000, 'model__subsample': 0.8}


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- id


In [7]:
from sklearn.model_selection import ParameterSampler, cross_val_score
from xgboost import XGBRegressor
from scipy.stats import uniform, randint
from tqdm import tqdm

# Define your base model and your search space
model = XGBRegressor(tree_method='hist', random_state=42)
param_dist = {
    'n_estimators'      : randint(1000, 5000),
    'learning_rate'     : uniform(0.05, 0.2),
    'max_depth'         : randint(4, 12),
    'subsample'         : uniform(0.7, 0.3),
    'colsample_bytree'  : uniform(0.3, 0.7),
    'colsample_bylevel' : uniform(0.3, 0.7),
    'reg_alpha'         : uniform(0.0, 3.0),
    'reg_lambda'        : uniform(0.0, 10.0),
}

# Sample N parameter settings
n_iter = 50
param_list = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

best_score = float('inf')
best_params = None

# Loop with a progress bar
for i, params in enumerate(tqdm(param_list, desc="Random search"), start=1):
    model.set_params(**params)
    # Evaluate via CV on your training split
    scores = cross_val_score(
        model, X_train, y_train,
        scoring='neg_mean_absolute_error', cv=3, n_jobs=-1
    )
    mae = -scores.mean()
    if mae < best_score:
        best_score = mae
        best_params = params

    # (Optional) print after each run:
    print(f"Run {i}/{n_iter} → MAE = {mae:.2f}")

print(f"Done {n_iter} runs. Best CV MAE = {best_score:.2f}")
print("Best params:", best_params)

Random search:   2%|▏         | 1/50 [00:22<18:18, 22.41s/it]

Run 1/50 → MAE = 65673.83


Random search:   4%|▍         | 2/50 [00:52<21:26, 26.80s/it]

Run 2/50 → MAE = 64995.60


Random search:   6%|▌         | 3/50 [01:13<18:54, 24.13s/it]

...

Run 48/50 → MAE = 62470.33


Random search:  98%|█████████▊| 49/50 [18:20<00:29, 29.56s/it]

Run 49/50 → MAE = 62668.85


Random search: 100%|██████████| 50/50 [18:35<00:00, 22.32s/it]

Run 50/50 → MAE = 64430.43
Done 50 runs. Best CV MAE = 61504.15
Best params: {'colsample_bylevel': 0.9289879319689553, 'colsample_bytree': 0.7245003417617129, 'learning_rate': 0.05183941032332593, 'max_depth': 9, 'n_estimators': 2496, 'reg_alpha': 1.9905053073241674, 'reg_lambda': 0.05061583846218687, 'subsample': 0.7482424154252496}
